In [ ]:
# CS485 HW2 AY22-2
# Created by Jack Summers

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout #standard layers needed for CNN
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator #allow us to create a generator to point at a directory and augment data on the fly
from tensorflow.keras.utils import to_categorical # for one-hot encoding labels
import keras

In [ ]:
#load data

x_train = "../input/HW2Images/x_train.npy/x_train.npy"
y_train = "../input/HW2Images/y_train.npy"
x_val = "../input/HW2Images/x_val.npy/x_val.npy"
y_val = "../input/HW2Images/y_val.npy"


# load arrays

x_train = np.load(x_train)
y_train = np.load(y_train)
x_val = np.load(x_val)
y_val = np.load(y_val)


#normalize with .astype per MAJ Ruiz's advice

x_train = x_train.astype('float32') / 255 #floating point values
x_val = x_val.astype('float32') / 255

#combine data, split later in generator
x_train = np.append(x_train, x_val,axis=0) #Learned how to append at https://numpy.org/doc/stable/reference/generated/numpy.append.html
y_train = np.append(y_train, y_val,axis=0)
y_train = to_categorical(y_train)


#print(len(y_train) - len(y_train)*0.2)
#print(151284/128)

#reshape vector IOT feed it into model and generator
x_train = x_train.reshape(189106, 28, 28, 1) #learned how to re shape at https://stackoverflow.com/questions/63279168/valueerror-input-0-of-layer-sequential-is-incompatible-with-the-layer-expect


In [ ]:
#call backs copied from my HW1 problem set:

callback_list = []

es_callback = EarlyStopping(monitor = 'val_acc', 
                           min_delta = .001, # after each epoch we want to see the val accuracy imporve by 0.001
                           patience = 20, # if min_delta not seen after 10 epochs, stop training
                           verbose = 1,
                           restore_best_weights = True) # restore weights of peak val accuracy of that epoch

loss_plat = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', # dont use this when using 'adam' optimizer
                                 factor = 0.1,
                                 patience = 20)

callback_list.append(es_callback)
#callback_list.append(loss_plat)

In [ ]:
# generators

#data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_datagen.fit(x_train)

train_gen = train_datagen.flow(x_train, y_train, batch_size=128, subset='training')
val_gen = train_datagen.flow(x_train, y_train, batch_size=37821, subset='validation') #all validation data at once



In [ ]:
#define model
                       

model = Sequential()
model.add(Conv2D(16, kernel_size = (3, 3), activation = 'relu', input_shape = (28,28,1))) #kernal size 3x3 standard and so is relu for CNN. input shape is 28 x 28 image with depth of 1 = grayscale
model.add(MaxPooling2D(2,2))
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(2, 2))
                                          
model.add(Flatten())

model.add(Dense(units = 128, activation = 'relu')) 
model.add(Dense(units = 8, activation = 'softmax')) # 8 options for classifcation. Use softmax for multi-class classification 
model.summary()

In [ ]:
#compile model
# "categorical crossentropy is almost always the loss function you should use for multi class classification"
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc']) 

In [ ]:
#train model with generator

history = model.fit(
    train_gen,
    steps_per_epoch= 1181, # 151284 training images / 128 batch size = 1181.90625
    epochs=1000,
    validation_data=val_gen,
    validation_steps=1, #batch size / 0.2*allData = approx. 1
    callbacks = callback_list)

In [ ]:
# plot results
model.save("./history.h5") # used https://www.tensorflow.org/tutorials/keras/save_and_load to learn how to save and load models
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training Acc.')
plt.plot(epochs, val_acc, 'b', label='Validation Acc.')
plt.title('Training vs. Validation Acc.')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training vs Validation Loss')
plt.legend()

plt.show()

In [ ]:
# load saved model 
#new_model = tf.keras.models.load_model('../input/models/0.55history.h5')

file = '../input/HW2xtest/x_test.npy' #input test data
   

#load and normalize test data 
test_data = np.load(file)
test_data = test_data.astype('float32') / 255
test_data = test_data.reshape(47280, 28, 28, 1)
#create predictions
predictions = model.predict(test_data)   #change if using loaded model

#output to csv
import csv
count = 0
with open('./HW2_predictions4.csv', mode = 'w') as file:
    row = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    row.writerow(['Id', 'Category']) # headers
    for item in predictions:
        row.writerow([count, np.argmax(item)]) # add row: count, index of predicted value
        count +=1